In [3]:
import osmnx as ox
import pandas as pd
import numpy as np
from data import csv_to_df
import matplotlib.pyplot as plt
import nvector as nv
import networkx as nx
from numpy import arctan2, sin, cos, sqrt, radians
from feature_eng import nearest_graph_data
from nvector import rad

In [4]:
df = csv_to_df('sample.csv')
g = ox.graph_from_point((37.979838, 23.733420), network_type='drive', dist=500) 
df = nearest_graph_data(df, g)
df

lat        lon    speed  lon_acc  lat_acc type  traveled_d  \
id time                                                                     
1  0.00  37.979838  23.733420  11.6261   0.0000  -0.0000  Car        1.81   
   0.04  37.979839  23.733419  11.6261  -0.0000  -0.0000  Car        1.81   
   0.08  37.979840  23.733418  11.6261   0.0000  -0.0000  Car        1.81   
   0.12  37.979841  23.733418  11.6261  -0.0000  -0.0000  Car        1.81   
   0.16  37.979842  23.733417  11.6261  -0.0000   0.0000  Car        1.81   
...            ...        ...      ...      ...      ...  ...         ...   
9  2.84  37.979802  23.733362  37.4951   0.0222  -0.0207  Car       29.88   
   2.88  37.979805  23.733359  37.4989   0.0148  -0.0195  Car       29.88   
   2.92  37.979808  23.733357  37.5021   0.0101  -0.0183  Car       29.88   
   2.96  37.979811  23.733354  37.5046   0.0022  -0.0171  Car       29.88   
   3.00  37.979814  23.733351  37.5060  -0.0096  -0.0159  Car       29.88   

         avg_speed  nearest_node  nearest_edge_start_node  \
id time                                                     
1  0.00  10.850992     250698924                250698924   
   0.04  10.850992     250698924                250698924   
   0.08  10.850992     250698924                250698924   
   0.12  10.850992     250698924                250698924   
   0.16  10.850992     250698924                250698924   
...            ...           ...                      ...   
9  2.84  35.851799     250698924                250698924   
   2.88  35.851799     250698924                250698924   
   2.92  35.851799     250698924                250698924   
   2.96  35.851799     250698924                250698924   
   3.00  35.851799     250698924                250698924   

         nearest_edge_end_node  edge_progress  
id time                                        
1  0.00              250698926       0.276860  
   0.04              250698926       0.277450  
   0.08              250698926       0.278040  
   0.12              250698926       0.278435  
   0.16              250698926       0.279026  
...                        ...            ...  
9  2.84              250698926       0.275281  
   2.88              250698926       0.277062  
   2.92              250698926       0.278619  
   2.96              250698926       0.280400  
   3.00              250698926       0.282181  

[534 rows x 12 columns]

In [5]:
def __calc_xtrack_dists(group, graph):
    start_node, end_node = group[['nearest_edge_start_node', 'nearest_edge_end_node']].iloc[0]
    lon_start, lat_start = graph.nodes[start_node]['x'],graph.nodes[start_node]['y']
    lon_end, lat_end = graph.nodes[end_node]['x'],graph.nodes[end_node]['y']
    lon_v, lat_v = group['lon'], group['lat']

    start_pt = nv.lat_lon2n_E(rad(lat_start), rad(lon_start))
    end_pt = nv.lat_lon2n_E(rad(lat_end), rad(lon_end))
    v_pt = nv.lat_lon2n_E(rad(lat_v), rad(lon_v))

    group['xtrack_dist'] = nv.cross_track_distance((start_pt,end_pt), v_pt)
    return group

In [6]:
df.groupby(['nearest_edge_start_node', 'nearest_edge_end_node'], as_index=False, group_keys=False) \
    .apply(__calc_xtrack_dists, g)

lat        lon    speed  lon_acc  lat_acc type  traveled_d  \
id time                                                                     
1  0.00  37.979838  23.733420  11.6261   0.0000  -0.0000  Car        1.81   
   0.04  37.979839  23.733419  11.6261  -0.0000  -0.0000  Car        1.81   
   0.08  37.979840  23.733418  11.6261   0.0000  -0.0000  Car        1.81   
   0.12  37.979841  23.733418  11.6261  -0.0000  -0.0000  Car        1.81   
   0.16  37.979842  23.733417  11.6261  -0.0000   0.0000  Car        1.81   
...            ...        ...      ...      ...      ...  ...         ...   
9  2.84  37.979802  23.733362  37.4951   0.0222  -0.0207  Car       29.88   
   2.88  37.979805  23.733359  37.4989   0.0148  -0.0195  Car       29.88   
   2.92  37.979808  23.733357  37.5021   0.0101  -0.0183  Car       29.88   
   2.96  37.979811  23.733354  37.5046   0.0022  -0.0171  Car       29.88   
   3.00  37.979814  23.733351  37.5060  -0.0096  -0.0159  Car       29.88   

         avg_speed  nearest_node  nearest_edge_start_node  \
id time                                                     
1  0.00  10.850992     250698924                250698924   
   0.04  10.850992     250698924                250698924   
   0.08  10.850992     250698924                250698924   
   0.12  10.850992     250698924                250698924   
   0.16  10.850992     250698924                250698924   
...            ...           ...                      ...   
9  2.84  35.851799     250698924                250698924   
   2.88  35.851799     250698924                250698924   
   2.92  35.851799     250698924                250698924   
   2.96  35.851799     250698924                250698924   
   3.00  35.851799     250698924                250698924   

         nearest_edge_end_node  edge_progress  xtrack_dist  
id time                                                     
1  0.00              250698926       0.276860     2.430654  
   0.04              250698926       0.277450     2.420627  
   0.08              250698926       0.278040     2.410601  
   0.12              250698926       0.278435     2.473078  
   0.16              250698926       0.279026     2.463052  
...                        ...            ...          ...  
9  2.84              250698926       0.275281    -4.023750  
   2.88              250698926       0.277062    -4.053829  
   2.92              250698926       0.278619    -4.011404  
   2.96              250698926       0.280400    -4.041483  
   3.00              250698926       0.282181    -4.071561  

[534 rows x 13 columns]